In [16]:
import sys

sys.path.append("../../../src/stong_core_cython")
sys.path.append("../../../cython")

In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import unrestricted_hartree_fock
import basis_data
import importlib
from scipy.ndimage import gaussian_filter1d
from matplotlib.backends.backend_pdf import PdfPages
from scipy.constants import physical_constants
from scipy.optimize import minimize

importlib.reload(unrestricted_hartree_fock)
importlib.reload(basis_data)

from atom import *
from unrestricted_hartree_fock import *
from basis_data import *
from matrix import *

hartree_to_ev_factor = physical_constants["Hartree energy in eV"][0]

In [9]:
df = pd.read_csv("../sto_ng/sto_3g.csv")
df_zetas = pd.read_csv("./zetas_4s.csv")

In [57]:
df

,exps_1,coefs_1s,exps_2,coefs_2s,coefs_2p,exps_3,coefs_3s,coefs_3p,coefs_3d,exps_4,coefs_4s,coefs_4p,coefs_4d
0,0.109987,0.445576,0.074772,0.694761,0.388124,0.053662,0.916705,0.484648,0.286555,0.040052,1.116951,0.544757,0.305929
1,0.407154,0.535362,0.229171,0.405600,0.608609,0.139076,0.217497,0.577749,0.655555,0.090982,0.055484,0.584370,0.668295
2,2.246014,0.153482,0.978367,-0.100302,0.158647,0.455947,-0.227739,0.004969,0.219772,0.233688,-0.329890,-0.128434,0.124780


In [49]:
atom_symbol = "H"
# zetas = df_zetas[atom_symbol]
atoms = [Atom(atom_symbol, np.array([0.0, 0.0, 0.0]))]
molecule = Molecule(atoms)
zetas = [1]
basis_data = {atom_symbol: create_basis_data(df, zetas, 1, 0)}

In [58]:
create_basis_data(df, zetas, 1, 0)

{'1s': {'exps': array([0.10998665, 0.40715402, 2.24601408]),
  'coefs': array([0.44557586, 0.5353617 , 0.15348232]),
  'lmn': array([0, 0, 0]),
  'zeta': 1}}

In [50]:
molecule

Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])

In [51]:
basis_data

{'H': {'1s': {'exps': array([0.10998665, 0.40715402, 2.24601408]),
   'coefs': array([0.44557586, 0.5353617 , 0.15348232]),
   'lmn': array([0, 0, 0]),
   'zeta': 1}}}

In [56]:
basis_functions = create_basis_function(molecule, basis_data)
get_Smat(basis_functions)

array([[1.]])

In [42]:
hartreefock = UnrestrictedHartreeFock(molecule, basis_data, max_iter=10000, tol=1e-10)
hartreefock.scf()
hartreefock.calculate_total_energy()
results = hartreefock.get_results()

Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])


In [43]:
results

{'energy': -0.4949421992363992,
 'P_up': array([[1.]]),
 'P_dn': array([[0.]]),
 'val_up': array([-0.4949422]),
 'val_dn': array([0.12987982]),
 'vec_up': array([[1.]]),
 'vec_dn': array([[1.]])}